# A2A Coordinator Agent

This example demonstrates a coordinator agent that routes tasks to specialized A2A agents. The coordinator discovers available agents, understands their capabilities through Agent Cards, and delegates work to the appropriate specialist.

**Start the A2A servers first** (in separate terminals):

```bash
# Terminal 1: Arithmetic agent on port 8000
uvicorn agentic_patterns.examples.a2a.example_a2a_server_1:app --host 0.0.0.0 --port 8000

# Terminal 2: Area calculator agent on port 8001
uvicorn agentic_patterns.examples.a2a.example_a2a_server_2:app --host 0.0.0.0 --port 8001
```

In [ ]:
import asyncio
import uuid

import httpx
import rich
from fasta2a.client import A2AClient
from fasta2a.schema import Message, TextPart
from agentic_patterns.core.agents import get_agent, run_agent


async def agent_card(base_url: str) -> dict:
    """Fetch the agent card from an A2A server."""
    async with httpx.AsyncClient() as http:
        response = await http.get(f"{base_url}/.well-known/agent-card.json")
        return response.json()


async def send_task(client: A2AClient, prompt: str) -> dict:
    """Send a message to an A2A agent and wait for the result."""
    message = Message(
        kind="message",
        role="user",
        parts=[TextPart(kind="text", text=prompt)],
        message_id=str(uuid.uuid4()),
    )
    response = await client.send_message(message=message)
    task_id = response["result"]["id"]

    while True:
        task_result = await client.get_task(task_id=task_id)
        state = task_result["result"]["status"]["state"]
        if state == "completed":
            return task_result
        elif state == "failed":
            raise Exception(f"Task failed: {task_result}")
        await asyncio.sleep(0.2)


def get_result_text(task_result: dict) -> str | None:
    """Extract the text result from a completed task."""
    for artifact in task_result["result"].get("artifacts", []):
        for part in artifact.get("parts", []):
            if part.get("kind") == "text":
                return part["text"]
    return None

## Clients

In [ ]:
SERVER_1_URL = "http://127.0.0.1:8000"
SERVER_2_URL = "http://127.0.0.1:8001"

client_1 = A2AClient(base_url=SERVER_1_URL)
client_2 = A2AClient(base_url=SERVER_2_URL)

clients = {SERVER_1_URL: client_1, SERVER_2_URL: client_2}

In [ ]:
cards = {}
clients_by_name = {}

for url, client in clients.items():
    card = await agent_card(url)
    rich.print(card)
    name = card["name"]
    cards[name] = card
    clients_by_name[name] = client

## Router agent

In [ ]:
def card_to_description(card: dict) -> str:
    """Convert agent card to a description string."""
    descr = f"{card['name']}: {card['description']}\n"
    for skill in card.get("skills", []):
        descr += f"  - {skill['name']}: {skill['description']}\n"
    return descr

In [ ]:
agent_descriptions = [card_to_description(card) for card in cards.values()]
agent_descriptions_str = "\n".join(agent_descriptions)

In [ ]:
system_prompt = f"""You route tasks to specialized agents.

Available agents:
{agent_descriptions_str}

NEVER perform calculations yourself. Always delegate to the appropriate agent.
Only invoke one agent at a time.
"""
print(system_prompt)

In [ ]:
async def route(agent_name: str, task_description: str) -> str | None:
    """Route a task to a specialized agent."""
    print(f"Routing to '{agent_name}': {task_description}")
    client = clients_by_name.get(agent_name)
    if not client:
        return f"Agent '{agent_name}' not found"
    result = await send_task(client, task_description)
    text = get_result_text(result)
    print(f"Result: {text}")
    return text

In [ ]:
coordinator = get_agent(system_prompt=system_prompt, tools=[route])

## Use the "Agentic system"

In [ ]:
message_history = []

prompt = "What is the sum of 40123456789 and 2123456789?"

agent_run, _ = await run_agent(
    agent=coordinator, prompt=prompt, message_history=message_history, verbose=True
)
message_history = agent_run.result.all_messages()

print(f"\nAnswer: {agent_run.result.output}")

In [ ]:
prompt = "From that result, subtract 246913578"

agent_run, _ = await run_agent(
    agent=coordinator, prompt=prompt, message_history=message_history, verbose=True
)
message_history = agent_run.result.all_messages()

print(f"\nAnswer: {agent_run.result.output}")

In [ ]:
prompt = "Calculate the area of a circle with radius equal to half that number"

agent_run, _ = await run_agent(
    agent=coordinator, prompt=prompt, message_history=message_history, verbose=True
)
message_history = agent_run.result.all_messages()

print(f"\nAnswer: {agent_run.result.output}")